In [9]:
import os
import numpy as np 
import pandas as pd
import glob
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
plt.style.use('ggplot')

In [2]:
root = r'C:\Users\Asura\Documents\Portfolio\projects\CORD-19 DATA'
metadata_path = f'{root}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

C:\Users\Asura\AppData\Local\Temp\ipykernel_10628\748881401.py:3: DtypeWarning: Columns (1,5,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(metadata_path, dtype={


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [3]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056660 entries, 0 to 1056659
Data columns (total 19 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   cord_uid          1056660 non-null  object 
 1   sha               373766 non-null   object 
 2   source_x          1056660 non-null  object 
 3   title             1056157 non-null  object 
 4   doi               656780 non-null   object 
 5   pmcid             389571 non-null   object 
 6   pubmed_id         498932 non-null   object 
 7   license           1056660 non-null  object 
 8   abstract          821116 non-null   object 
 9   publish_time      1054846 non-null  object 
 10  authors           1032791 non-null  object 
 11  journal           969338 non-null   object 
 12  mag_id            0 non-null        float64
 13  who_covidence_id  482935 non-null   object 
 14  arxiv_id          14249 non-null    object 
 15  pdf_json_files    373766 non-null   object 
 16  

In [4]:
# all_json = !ls $root_path/document_parses/pdf_json
pdf_json_path = os.path.join(root, "document_parses/pdf_json")
all_json = os.listdir(pdf_json_path)
all_json_path = []
for path in all_json:
    json_path = os.path.join(pdf_json_path, path)
    json_path = json_path.replace('\\', '/')
    all_json_path.append(json_path)
all_json_path[:5]

['C:/Users/Asura/Documents/Portfolio/projects/CORD-19 DATA/document_parses/pdf_json/0000028b5cc154f68b8a269f6578f21e31f62977.json',
 'C:/Users/Asura/Documents/Portfolio/projects/CORD-19 DATA/document_parses/pdf_json/0000b6da665726420ab8ac9246d526f2f44d5943.json',
 'C:/Users/Asura/Documents/Portfolio/projects/CORD-19 DATA/document_parses/pdf_json/0000b93c66f991236db92dc16fa6db119b27ca12.json',
 'C:/Users/Asura/Documents/Portfolio/projects/CORD-19 DATA/document_parses/pdf_json/0000fcce604204b1b9d876dc073eb529eb5ce305.json',
 'C:/Users/Asura/Documents/Portfolio/projects/CORD-19 DATA/document_parses/pdf_json/000122a9a774ec76fa35ec0c0f6734e7e8d0c541.json']

In [29]:
with open(all_json_path[1]) as file:
    first_entry = json.load(file)
    print(json.dumps(first_entry, indent=2))
    print(first_entry.keys())

{
  "paper_id": "0000b6da665726420ab8ac9246d526f2f44d5943",
  "metadata": {
    "title": "The cell phone vibration test: A telemedicine substitute for the tuning fork test",
    "authors": [
      {
        "first": "Alex",
        "middle": [
          "Yang"
        ],
        "last": "Ba",
        "suffix": "",
        "affiliation": {
          "laboratory": "",
          "institution": "Uniformed Services University of the Health Sciences",
          "location": {
            "settlement": "Bethesda",
            "region": "Maryland",
            "country": "USA"
          }
        },
        "email": ""
      },
      {
        "first": "|",
        "middle": [],
        "last": "Robert",
        "suffix": "",
        "affiliation": {},
        "email": ""
      },
      {
        "first": "J",
        "middle": [],
        "last": "Lewis",
        "suffix": "",
        "affiliation": {
          "laboratory": "",
          "institution": "Walter Reed National Military Medical C

In [16]:
class Reader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstarct = []
            self.body_text = []
            # extract all text from abstract
            for abstarct in content['abstract']:
                self.abstarct.append(abstarct['text'])
            # extract all text from body_text
            for body_text in content['body_text']: 
                self.body_text.append(body_text['text'])
            self.abstarct = '\n'.join(self.abstarct)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstarct[:300]}... {self.body_text[:300]}...'
first_entry = Reader(all_json_path[0])
print(first_entry)

0000028b5cc154f68b8a269f6578f21e31f62977: ... According to current live statistics at the time of editing this letter, Russia has been the third country in the world to be affected by COVID-19 with both new cases and death rates rising. It remains in a position of advantage due to the later onset of the viral spread within the country since the...


In [8]:
all_json = list()
for idx, path in tqdm(enumerate(all_json_path), total=len(all_json_path)):
    try:
        content = Reader(path)
    except Exception as e:
        continue
    if len(content.body_text) == 0:
        continue
    all_json.append(all_json_path[idx])

len(all_json)


100%|██████████| 401212/401212 [37:45<00:00, 177.07it/s]


401212

In [10]:
random.seed(9)
all_json = random.sample(all_json, 100000)

In [24]:
def put_breaks(text,length):
    words = text.split(' ')
    data = ""
    total_chars = 0

    for word in words:
        total_chars += len(word)
        if total_chars > length:
            data += '<br>'
            total_chars = 0
        data += word + ' '
    return data


In [25]:
text = "This is a sample text that will be broken into lines based on the specified length."
result = put_breaks(text, 20)
print(result)

This is a sample text <br>that will be broken into <br>lines based on the specified <br>length. 


In [33]:
temp_dict = {'paper_id': [],'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in tqdm(enumerate(all_json), total=len(all_json)):
    try:
        content = Reader(entry)
    except Exception as e:
        continue
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha']==content.paper_id]
    if len(meta_data) == 0:
        continue
    temp_dict['paper_id'].append(content.paper_id)
    temp_dict['abstract'].append(content.abstarct)
    temp_dict['body_text'].append(content.body_text)

    if len(content.abstarct) == 0:
        temp_dict['abstract_summary'].append("Not provided.")
    elif len(content.abstarct.split(' ')) > 100:
        words = content.abstarct.split(' ')[:100]
        summary = put_breaks(' '.join(words), 40)
        temp_dict['abstract_summary'].append(summary+'...')
    else:
        summary = put_breaks(content.abstarct, 40)
        temp_dict['abstract_summary'].append(summary)
    
    try:
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            temp_dict['authors'].append(put_breaks(". ".join(authors),40) )
        else:
            temp_dict['authors'].append(". ".join(authors))
    except Exception as e:
        temp_dict['authors'].append(meta_data['authors'].values[0])
    
    try:
        title = put_breaks(meta_data['title'].values[0], 40)
        temp_dict['title'].append(title)
    except Exception as e:
        temp_dict['title'].append(meta_data['title'].values[0])
    
    temp_dict['doi'].append(meta_data['doi'].values[0])
    temp_dict['journal'].append(meta_data['journal'].values[0])

df_covid = pd.DataFrame(temp_dict, columns=['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

100%|██████████| 100000/100000 [46:19<00:00, 35.98it/s]


,paper_id,doi,abstract,body_text,authors,title,journal,abstract_summary
0,7cd1a2cfeec13831d59177596a41051ec9a30ebc,10.1007/s13142-011-0040-4,This commentary describes key observations and...,Conducting research in Asian and African setti...,"Baumann, Linda Ciofu",Insights on conducting research in <br>low-res...,Transl Behav Med,This commentary describes key observations <br...
1,597a9eda59c4d2cb8f37391cc54bd98751f2ad25,10.1186/s12917-020-02377-5,"Background: Glaesserella parasuis, the causati...",Glaesserella parasuis is a Gram-negative bacte...,"Hau, Samantha J.. Luan, Shi-Lu. Loving, <br>...",Evaluation of the recombinant proteins RlpB <b...,BMC Vet Res,"Background: Glaesserella parasuis, the <br>cau..."
2,2e8227b804b3b030ba84de665f65a1ee8c0f1226,10.1101/2020.04.27.20081836,"Social distancing measures, with varying degre...",Over nine hundred thousand Americans are confi...,"Wagner, A. B.. Hill, E. L.. Ryan, S. E.. Su...",Social Distancing Has Merely Stabilized <br>CO...,NaN,"Social distancing measures, with varying <br>d..."
3,3e621661de7ef0a955156ffcb894aaf6ac60f039,10.1016/j.soc.2020.06.009,Immunotherapy Targeted therapy T cells Adoptiv...,Over the past decade there have been dramatic ...,"Sepesi, Boris. Cascone, Tina. Chun, Stephen ...",Emerging Therapies in Thoracic <br>Malignancie...,Surg Oncol Clin N Am,Immunotherapy Targeted therapy T cells <br>Ado...
4,e28730591279832cd6fa9fe6ec24a1512182c5c5,10.1080/16549716.2021.1903222,Background: Official development assistance (O...,Development Assistance for Health (DAH) is one...,"Nomura, Shuhei. Sakamoto, Haruka. Ishizuka, ...",Tracking sectoral allocation of official <br>d...,Global health action,Background: Official development assistance <b...


In [34]:
df_covid.to_csv('covid19_final.csv')

In [35]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87189 entries, 0 to 87188
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   paper_id          87189 non-null  object
 1   doi               84199 non-null  object
 2   abstract          87189 non-null  object
 3   body_text         87189 non-null  object
 4   authors           86505 non-null  object
 5   title             87189 non-null  object
 6   journal           79430 non-null  object
 7   abstract_summary  87189 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB
